In [25]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [4]:
import matplotlib.pyplot as plt

In [15]:
def encode(series): 
    return pd.get_dummies(series.astype(str))

In [16]:
train_x = pd.read_csv('../../big datasets/drugml/x_train_res.csv')
train_y = pd.read_csv('../../big datasets/drugml/y_train_res.csv')
test_x = pd.read_csv('../../big datasets/drugml/x_test.csv')
test_y = pd.read_csv('../../big datasets/drugml/y_test.csv')

In [17]:
from sklearn.preprocessing import StandardScaler
xscaler = StandardScaler().fit(train_x)
train_x = xscaler.transform(train_x)
testscaler = StandardScaler().fit(test_x)
test_x = testscaler.transform(test_x)


C:\Users\ellie\Miniconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ellie\Miniconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ellie\Miniconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ellie\Miniconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [18]:
x_train = pd.DataFrame(train_x)
train_y = pd.DataFrame(train_y)
x_test = pd.DataFrame(test_x)
test_y = pd.DataFrame(test_y)

In [19]:
#x_train = x_train.drop(['Unnamed: 0'], axis=1)

train_y = train_y.drop(['Unnamed: 0'], axis=1)
y_train = encode(train_y)
#y_train = encode(train_y)
#x_test = test_x.drop(['Unnamed: 0'], axis=1)
test_y = test_y.drop(['Unnamed: 0'], axis=1)
y_test = encode(test_y)
#y_test = encode(test_y)

In [34]:
x_train['Class-1'] = y_train.iloc[:,0]
x_train['Class-2'] = y_train.iloc[:,1]
x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,272,273,274,275,276,277,278,279,Class-1,Class-2
0,-1.730986,0.921982,-0.157973,-1.459674,0.334478,0.369859,0.428547,1.187530,-0.313013,-0.179779,...,0.877483,0.613616,-0.820161,-0.338000,0.806019,0.234209,1.225323,-1.425786,0,1
1,-1.728855,0.855233,-0.157973,-1.165434,-0.087522,0.817984,1.128209,0.677041,2.486413,-0.179779,...,0.937737,1.266806,0.683059,-0.337961,0.963520,0.346804,1.350914,-0.382318,0,1
2,-1.726725,-0.490926,-0.157973,-0.092644,-0.689763,0.821746,0.428547,0.166552,-0.313013,-0.179779,...,0.924800,0.617204,-0.217306,-0.337907,0.784516,0.449160,0.220684,-0.429355,0,1
3,-1.724595,1.783538,-0.157973,-2.115771,1.673549,0.956781,2.527533,-0.854426,-0.313013,-0.179779,...,0.729878,0.575758,-0.049837,-0.337926,1.128384,0.467271,0.824613,-0.956659,0,1
4,-1.722464,1.576920,-0.157973,-0.512310,-0.629174,0.681700,0.428547,0.166552,-0.313013,-0.179779,...,1.639030,2.540884,1.382322,-0.338042,0.779403,0.436493,0.351273,-0.689293,0,1


In [35]:
from GAN_171103 import *

ModuleNotFoundError: No module named 'GAN_171103'

In [20]:
def generator(z,reuse=None):
    with tf.variable_scope('gen', reuse=reuse):
        hidden1=tf.layers.dense(inputs=z, units=60, activation=tf.nn.leaky_relu)
        hidden2=tf.layers.dense(inputs=hidden1, units=60, activation=tf.nn.leaky_relu)
        output=tf.layers.dense(inputs=hidden2, units=280, activation=tf.nn.tanh)
        
        return output

def discriminator(x, reuse=None):
    with tf.variable_scope('dis', reuse=reuse):
        hidden1=tf.layers.dense(inputs=x, units=60, activation=tf.nn.leaky_relu)
        hidden2=tf.layers.dense(inputs=hidden1, units=60, activation=tf.nn.leaky_relu)
        logits=tf.layers.dense(hidden2, units=280)
        output=tf.sigmoid(logits)
        
        return output, logits
    ### discriminator should be similar to what i have in the nn

In [21]:
tf.reset_default_graph()

real_molecules=tf.placeholder(tf.float32, shape=[None,280])
z = tf.placeholder(tf.float32, shape=[None,280])

G = generator(z)
D_output_real,D_logits_real = discriminator(real_molecules)
D_output_fake,D_logits_fake = discriminator(G, reuse=True)

In [22]:
def loss_func(logits_in, labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in, labels=labels_in))
                          
D_real_loss = loss_func(D_logits_real,tf.ones_like(D_logits_real)*0.9)
D_fake_loss = loss_func(D_logits_fake, tf.zeros_like(D_logits_real))
D_loss=D_real_loss + D_fake_loss
                          
G_loss = loss_func(D_logits_fake, tf.ones_like(D_logits_fake))

In [23]:
lr=0.001

tvars=tf.trainable_variables()  #returns all variables created(the two variable scopes) and makes trainable true
d_vars=[var for var in tvars if 'dis' in var.name]
g_vars=[var for var in tvars if 'gen' in var.name]

D_trainer=tf.train.AdamOptimizer(lr).minimize(D_loss,var_list=d_vars)
G_trainer=tf.train.AdamOptimizer(lr).minimize(G_loss,var_list=g_vars)

batch_size=100
epochs=100
init=tf.global_variables_initializer()

In [30]:
samples = []

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        #num_batches=mnist.train.num_examples//batch_size
        num_batches = int(len(x_train)/batch_size)
        x_batches = np.array_split(x_train, num_batches)
        y_batches = np.array_split(y_train, num_batches)
        for i in range(num_batches):
            #batch=mnist.train.next_batch(batch_size)
            batch_x, batch_y = x_batches[i], y_batches[i]
            
            batch_mols=batch_x[0] + batch_y[0] #reshape((batch_size,280))
            batch_mols=batch_mols*2-1
            batch_z=np.random.uniform(-1,1,size=(batch_size,280))
            _=sess.run(D_trainer,feed_dict={real_molecules:batch_mols,z:batch_z})
            _=sess.run(G_trainer,feed_dict={z:batch_z})
            
        print("on epoch{}".format(epoch))
        
        sample_z=np.random.uniform(-1,1,size=(1,100))
        gen_sample=sess.run(generator(z,reuse=True),feed_dict={z:sample_z})
        
        samples.append(gen_sample)
  

KeyError: 0

In [ ]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

def generator(Z,hsize=[16, 16],reuse=False):
    with tf.variable_scope("GAN/Generator",reuse=reuse):
        h1 = tf.layers.dense(Z,hsize[0],activation=tf.nn.leaky_relu)
        h2 = tf.layers.dense(h1,hsize[1],activation=tf.nn.leaky_relu)
        out = tf.layers.dense(h2,2)

    return out

def discriminator(X,hsize=[16, 16],reuse=False):
    with tf.variable_scope("GAN/Discriminator",reuse=reuse):
        h1 = tf.layers.dense(X,hsize[0],activation=tf.nn.leaky_relu)
        h2 = tf.layers.dense(h1,hsize[1],activation=tf.nn.leaky_relu)
        h3 = tf.layers.dense(h2,2)
        out = tf.layers.dense(h3,1)

    return out, h3


X = tf.placeholder(tf.float32,[None,2])
Z = tf.placeholder(tf.float32,[None,2])

G_sample = generator(Z)
r_logits, r_rep = discriminator(X)
f_logits, g_rep = discriminator(G_sample,reuse=True)

disc_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=r_logits,labels=tf.ones_like(r_logits)) + tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,labels=tf.zeros_like(f_logits)))
gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits,labels=tf.ones_like(f_logits)))

gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Generator")
disc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="GAN/Discriminator")

gen_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(gen_loss,var_list = gen_vars) # G Train step
disc_step = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(disc_loss,var_list = disc_vars) # D Train step



# sess = tf.Session(config=config)
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)

batch_size = 256
nd_steps = 10
ng_steps = 10

x_plot = sample_data(n=batch_size)

f = open('loss_logs.csv','w')
f.write('Iteration,Discriminator Loss,Generator Loss\n')

for i in range(10001):
    X_batch = sample_data(n=batch_size)
    Z_batch = sample_Z(batch_size, 2)

    for _ in range(nd_steps):
        _, dloss = sess.run([disc_step, disc_loss], feed_dict={X: X_batch, Z: Z_batch})
    rrep_dstep, grep_dstep = sess.run([r_rep, g_rep], feed_dict={X: X_batch, Z: Z_batch})

    for _ in range(ng_steps):
        _, gloss = sess.run([gen_step, gen_loss], feed_dict={Z: Z_batch})

    rrep_gstep, grep_gstep = sess.run([r_rep, g_rep], feed_dict={X: X_batch, Z: Z_batch})

    print "Iterations: %d\t Discriminator loss: %.4f\t Generator loss: %.4f"%(i,dloss,gloss)
    if i%10 == 0:
        f.write("%d,%f,%f\n"%(i,dloss,gloss))
        
        #### training data
import numpy as np
import math


def get_y(x):
    return 10 + x*x


def sample_data(n=10000, scale=100):
    data = []

    x = scale*(np.random.random_sample((n,))-0.5)

    for i in range(n):
        yi = get_y(x[i])
        data.append([x[i], yi])

    return np.array(data)